# Predicting Weather Using Hidden Markov Model (HMM)

This notebook demonstrates how to predict weather states from weather data using a Hidden Markov Model (HMM).

## 1. Import Required Libraries

We will use numpy, pandas, matplotlib for data handling and visualization, and hmmlearn for HMM implementation.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm

## 2. Load and Explore Weather Data

Load your weather dataset using pandas. Display basic statistics and visualize the data to understand its structure.

In [ ]:
# Load the weather dataset
# Replace 'weather.csv' with your actual data file path
data = pd.read_csv('weather.csv')

# Display first few rows
print(data.head())

# Display basic statistics
print(data.describe())

# Visualize the distribution of weather states
plt.figure(figsize=(8,4))
data['Weather'].value_counts().plot(kind='bar')
plt.title('Distribution of Weather States')
plt.xlabel('Weather State')
plt.ylabel('Count')
plt.show()

## 3. Preprocess Data for HMM

Clean the data, handle missing values, and select relevant features for HMM modeling.

In [ ]:
# Check for missing values
print(data.isnull().sum())

# Drop rows with missing values (or handle appropriately)
data = data.dropna()

# Select relevant features (e.g., Temperature, Humidity, Weather)
features = ['Temperature', 'Humidity', 'Weather']
data = data[features]

## 4. Encode Weather States

Convert categorical weather states (e.g., 'Sunny', 'Rainy') into numerical labels suitable for HMM.

In [ ]:
# Encode weather states as integers
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['Weather_encoded'] = le.fit_transform(data['Weather'])

# Display mapping
print(dict(zip(le.classes_, le.transform(le.classes_))))

## 5. Train Hidden Markov Model

Use hmmlearn's GaussianHMM or MultinomialHMM to fit the model to the encoded weather data.

In [ ]:
# Prepare observation sequence for HMM
# For MultinomialHMM, observations should be integers (encoded weather states)
observations = data['Weather_encoded'].values.reshape(-1, 1)

# Define and train the HMM
model = hmm.MultinomialHMM(n_components=len(le.classes_), n_iter=100, random_state=42)
model.fit(observations)

# Print model parameters
print("Transition matrix:\n", model.transmat_)
print("Emission probabilities:\n", model.emissionprob_)

## 6. Predict Weather States

Use the trained HMM to predict hidden weather states on test data or future sequences.

In [ ]:
# Predict hidden states for the observed sequence
hidden_states = model.predict(observations)

# Decode predicted states to original labels
predicted_weather = le.inverse_transform(hidden_states)

# Add predictions to the dataframe
data['Predicted_Weather'] = predicted_weather

# Display first few predictions
print(data[['Weather', 'Predicted_Weather']].head())

## 7. Visualize Predictions

Plot the actual vs predicted weather states to evaluate model performance.

In [ ]:
# Plot actual vs predicted weather states
plt.figure(figsize=(12,5))
plt.plot(data.index, data['Weather_encoded'], label='Actual', marker='o')
plt.plot(data.index, hidden_states, label='Predicted', marker='x')
plt.title('Actual vs Predicted Weather States')
plt.xlabel('Time Index')
plt.ylabel('Weather State (Encoded)')
plt.legend()
plt.show()